https://www.nhs.uk/conditions/

In [1]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# 定义随机的useragent
from fake_useragent import UserAgent
ua = UserAgent()

# 设置代理ip
ipdata = pd.read_excel('D:/Intern_Project/代理ip的地址2000.xlsx')
ipdata.fillna("",inplace = True)

proxieslist = []

for i in ipdata.index.values:
    line = ipdata.loc[i,['HTTP']].to_dict()
    proxieslist.append(line)
    
# randomproxy = random.choice(proxieslist)

# 设置消息头
headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
}

params = {
    'ie': 'UTF-8',
    'wd': 'Python'
}


In [3]:
def changeparams():
    headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }
    randomproxy = random.choice(proxieslist)
    #  访问百度服务器，获取cookie
    res = requests.get('https://www.nhs.uk', headers=headers_pc, proxies= randomproxy,)
    # 将cookieJar数据转化为字典，s
    cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
    return headers_pc, randomproxy, cookie_dict

In [4]:
def get_response(url, headers_pc, randomproxy, cookie_dict):
    try:
        requests.adapters.DEFAULT_RETRIES = 12 # 增加重连次数
        s = requests.session()
        s.keep_alive = False # 关闭多余连接
        
        r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, cookies=cookie_dict, timeout=10)
        r.encoding = 'utf-8'
        return r.text
    except Exception as e:
        time.sleep(5)
        print(e)

    return None

# 1、先获取术语链接列表

In [13]:
def get_term_url(url, tag, headers_pc, randomproxy, cookie_dict):
    results = []
    try:

        html = get_response(url, headers_pc, randomproxy, cookie_dict)
        
        if tag == 'health':
            with open('D:/Intern_Project/7_nhs/health术语列表页.html', 'w+', encoding='utf-8') as f:
                 f.write(html)
        if tag == 'medicines':      
            with open('D:/Intern_Project/7_nhs/medicines术语列表页.html', 'w+', encoding='utf-8') as f:
                 f.write(html)

        selector = Selector(text=html)
        termUrl_list = selector.xpath(
            "//ul[@class='nhsuk-list nhsuk-list--border']/li/a")
        #print(len(termUrl_list))
        for termUrl in termUrl_list:
            if tag == 'health':
                url = 'https://www.nhs.uk' + termUrl.xpath('./@href').extract()[0]
            if tag == 'medicines':
                url = 'https://www.nhs.uk' + termUrl.xpath('./@href').extract()[0]
            term = termUrl.xpath('./text()').extract()[0].strip()
            # print(term)
            tmp = {
                'term': term,
                'url': url
            }
            results.append(tmp)
            # time.sleep()
    except Exception as e:
        print(e)
    return results

In [14]:
# 获取health_termlist
headers_pc, randomproxy, cookie_dict = changeparams()

health_url = f'https://www.nhs.uk/conditions/'
health_results = get_term_url(health_url,'health', headers_pc, randomproxy, cookie_dict)
with open('D:/Intern_Project/7_nhs/health术语链接列表.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(health_results, indent=2, ensure_ascii=False))

In [15]:
# 获取medicines_termlist
headers_pc, randomproxy, cookie_dict = changeparams()
medicines_url = f'https://www.nhs.uk/medicines/'
medicines_results = get_term_url(medicines_url,'medicines', headers_pc, randomproxy, cookie_dict)
with open('D:/Intern_Project/7_nhs/medicines术语链接列表.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(medicines_results, indent=2, ensure_ascii=False))